In [ ]:
import tensorflow as tf
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np

train_files_healthy = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Train\\Train\\Healthy"
train_files_powdery = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Train\\Train\\Powdery"
train_files_rust = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Train\\Train\\Rust"

test_files_healthy = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Test\\Test\\Healthy"
test_files_powdery = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Test\\Test\\Powdery"
test_files_rust = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Test\\Test\\Rust"

valid_files_healthy = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Validation\\Validation\\Healthy"
valid_files_powdery = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Validation\\Validation\\Powdery"
valid_files_rust = "C:\\Users\\suhas\\BACKEND\\plantdataset\\Validation\\Validation\\Rust"

def total_files(folder_path):
    num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    return num_files

print("training set")
print("Number of healthy leaf images in training set:", total_files(train_files_healthy))
print("Number of powdery leaf images in training set:", total_files(train_files_powdery))
print("Number of rusty leaf images in training set:", total_files(train_files_rust))

print("\ntest set")
print("Number of healthy leaf images in test set:", total_files(test_files_healthy))
print("Number of powdery leaf images in test set:", total_files(test_files_powdery))
print("Number of rusty leaf images in test set:", total_files(test_files_rust))

print("\nvalidation")
print("Number of healthy leaf images in validation set:", total_files(valid_files_healthy))
print("Number of powdery leaf images in validation set:", total_files(valid_files_powdery))
print("Number of rusty leaf images in validation set:", total_files(valid_files_rust))

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory("C:\\Users\\suhas\\BACKEND\\plantdataset\\Train\\Train",
                                              target_size=(225, 225),
                                              batch_size=32,
                                              class_mode='categorical')

validation_gen = test_datagen.flow_from_directory('C:\\Users\\suhas\\BACKEND\\plantdataset\\Validation\\Validation',
                                                  target_size=(225, 225),
                                                  batch_size=32,
                                                  class_mode='categorical')

model = Sequential([
    Conv2D(32, (3, 3), input_shape=(225, 225, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_gen,
                    batch_size=16,
                    epochs=5,
                    validation_data=validation_gen,
                    validation_batch_size=16
                    )

model.save("model.h5")

def preprocess_image(image_path, target_size=(225, 225)):
    img = load_img(image_path, target_size=target_size)
    x = img_to_array(img)
    x = x.astype('float32') / 255.
    x = np.expand_dims(x, axis=0)
    return x

x = preprocess_image("C:\\Users\\suhas\\BACKEND\\plantdataset\\Test\\Test\\Powdery\\9ed17032dce2f984.jpg")
predictions = model.predict(x)

labels = train_gen.class_indices
labels = {v: k for k, v in labels.items()}
predicted_label = labels[np.argmax(predictions)]

print("Predicted Label:", predicted_label)

# Provide explanation for predicted label
if predicted_label == 'Healthy':
    print("The plant is healthy.")
    print("<a href='https://agri.gujarat.gov.in/index.htm'>Know More</a>")
elif predicted_label == 'Rust':
    print("The plant has rust disease. Rust diseases are amongst the most common fungal diseases of garden plants.")
    print("<a href='https://agri.gujarat.gov.in/index.htm'>Know More</a>")
else:
    print("The plant has powdery mildew disease. Powdery mildew is a fungal disease of the foliage, stems, and occasionally flowers and fruit.")
    print("<a href='https://agri.gujarat.gov.in/index.htm'>Know More</a>")


training set
Number of healthy leaf images in training set: 458
Number of powdery leaf images in training set: 430
Number of rusty leaf images in training set: 434

test set
Number of healthy leaf images in test set: 50
Number of powdery leaf images in test set: 50
Number of rusty leaf images in test set: 50

validation
Number of healthy leaf images in validation set: 20
Number of powdery leaf images in validation set: 20
Number of rusty leaf images in validation set: 20
Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Epoch 1/5
